In [1]:
from custom_filter import * 

In [7]:
f = f2
# --------------------------------------------------
base_dir = '../../outputs'
import json 
import os 
import numpy as np 
from omegaconf import OmegaConf 

results_in_directories = []
for root, dirs, files in os.walk(base_dir):
    if 'config.yaml' in files:
        flags = OmegaConf.load(os.path.join(root, 'config.yaml'))
        if not f(flags):
            continue
        print(flags)
        result = json.load(open(os.path.join(root, 'evaluation.json'), 'rb'))
        results_in_directories.append([flags, result])
        print(list(result.keys())[:10])
print(len(results_in_directories))
N_samples = 50000

{'encoder': 'resnet18', 'data_path': '/data/ImageNet1k', 'bbox_path': '/data/ILSVRC2012_bbox_val', 'input_attrib': 'grad', 'method': 'cls', 'device': 'cuda:0', 'layer_ratio': 0.9, 'alpha': None, 'quantile': 0.1, 'p_value_threshold': 0.05, 'variance_conservation': False, 'exact_variance': True, 'gamma_infinity': True, 'success': True}
[]
1


In [5]:
from tqdm import tqdm 
def custom_flag_condition(flag):
    name = f"layer_ratio_{flag.layer_ratio}_quatile_{flag.quantile}_variance_conservation_{flag.variance_conservation}"
    condition = True 
    return name, condition 

condition_results = {}
for flag, results in tqdm(results_in_directories): 
    name, condition = custom_flag_condition(flag)
    if not condition :
        continue
    eval_results = {
        'morf_aopc' : {},
        'morf_lodds' :{},
        'morf_fracdiff' : {},
        'morf_acc' :{},
        'lerf_aopc' :{},
        'lerf_lodds' :{},
        'lerf_fracdiff' :{},
        'lerf_acc' :{},
        'mu_in':[],
        'mu_in_w':[],
    }
    assert name not in condition_results, f"{name} is already in condition_results."
    reduced_eval_results =  {k:{} for k in eval_results.keys()}

    for idx in range(N_samples):
        for k, v in results[str(idx)].items():
            if k in ['morf', 'lerf']:
                for metric, ratio_results in v.items():
                    for ratio, value in ratio_results.items():
                        if ratio not in eval_results[f'{k}_{metric}']:
                            eval_results[f'{k}_{metric}'][float(ratio)] = []
                            reduced_eval_results[f'{k}_{metric}'][float(ratio)] = 0
                        eval_results[f'{k}_{metric}'][float(ratio)].append(float(value))
            elif k in ['mu_in_w', 'mu_in']:
                eval_results[k].append(float(v))
                
    for k, v in eval_results.items():
        if k in ['mu_in', 'mu_in_w'] :
            reduced_eval_results[f"{k}_mean"] = np.mean(v)
        else:
            for ratio, value in v.items():
                reduced_eval_results[f"{k}"][ratio] = np.mean(value)
            reduced_eval_results[f"{k}_ratio_mean"] = np.mean(list(reduced_eval_results[k].values()))
    condition_results[f'{name}'] = reduced_eval_results
    metric_names = [ k for k in  reduced_eval_results.keys() if 'mean' in k] 
    print(f'{name}')

  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: '0'

In [ ]:
import re 
for k in metric_names:
    sub_name = re.sub('_mean', "", k)
    sub_name = re.sub('_ratio', "", sub_name)
    print(sub_name, end= " | ")
    
print()
print("|", end=" ")
for k in metric_names:
    print("---", end= " | ")
print()
for name, reduced_eval_results in condition_results.items():
    print(name, end=" | ")
    for k in metric_names:
        print(f"{reduced_eval_results[k]:.3f}", end=" | ")
    print()

NameError: name 'metric_names' is not defined